# LAB 01: LAYOFFS 2023

## Mô tả đồ án:
- Từ nguồn dữ liệu lấy được từ Kaggle về vấn đề sa thải nhân viên ở các công ty trên toàn thế giới, đồ án này nhằm mục đích phân tích dữ liệu đã có để có được cái nhìn khách quan hơn về vấn đề này.

---

### Import các thư viện

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

---

### <span style="color:yellow"> A. Thu thập dữ liệu</span>


#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

Sau dịch covid 19, các công ty công nghệ trên toàn cầu đang đối mặt với sự suy thoái kinh tế. Sự chậm lại của chi tiêu từ người tiêu dùng, lãi suất cao hơn từ các ngân hàng trung ương và sức mạnh của đồng đô la ngoại tệ đang gợi ý về khả năng xảy ra suy thoái kinh tế và các công ty công nghệ đã bắt đầu **cắt giảm nhân sự**.

#### Chủ đề

Phân tích tình trạng hỗn loạn công nghệ gần đây thông qua số liệu về việc sa thải nhân sự, từ đó rút ra được những thông tin hữu ích.

#### Nguồn
Dữ liệu được lấy trên Kaggle dưới file csv, đường link dẫn đến  [Ở đây](https://www.kaggle.com/datasets/swaptr/layoffs-2022?resource=download)

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

#### Đọc dữ liệu

In [2]:
df=pd.read_csv('./datasets/layoffs.csv')
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,Atlassian,Sydney,Other,500.0,0.05,2023-03-06,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,2023-03-06,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,2023-03-06,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,2023-03-06,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,2023-03-03,Unknown,Brazil,788.0


---

### <span style="color:yellow"> B. Khám phá dữ liệu</span>


#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

In [3]:
# df['stage']

- Mỗi dòng chứa dữ liệu về một công ty và các thông tin liên quan đến công ty đó như: tên công ty, nơi đặt trụ sở chính, loại ngành nghề, số nhân viên bị sa thải, tỷ lệ sa thải, ngày công bố, giai đoạn của công ty, quốc gia công ty hoạt động, số vốn huy động bởi công ty.

- ví dụ: dòng 1: doanh nghiệp SiriusXM có trụ sở tại New York City và hoạt động trong ngành truyền thông, đã sa thải 475 nhân viên, chiếm 8% tổng số nhân viên của họ tính tới ngày 6 tháng 3 năm 2023. Công ty cũng đang ở giai đoạn Post-IPO và có trụ sở tại Hoa Kỳ. Họ đã huy động được số vốn 525 triệu đô la.

- Không có tình trạng các dòng có ý nghĩa khác nhau.

#### Mỗi cột có ý nghĩa gì?

## Thông tin của mỗi cột:
- Company: Tên của công ty
- Location: Nơi đặt trụ sở chính của công ty
- Industry: Loại ngành mà công ty hoạt động
- Total_laid_off: Tổng số nhân viên bị sa thải bởi công ty
- Percentage_laid_off: Tỷ lệ phần trăm nhân viên bị sa thải bởi công ty
- Date: Ngày công bố thông tin về việc sa thải
- Stage: Giai đoạn của công ty (Post-IPO, Series B, Unknown)
- Country: Quốc gia nơi công ty đặt trụ sở
- Funds_raised: Tổng số vốn do công ty huy động (tính bằng triệu đô la)

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [4]:
df.dtypes

company                 object
location                object
industry                object
total_laid_off         float64
percentage_laid_off    float64
date                    object
stage                   object
country                 object
funds_raised           float64
dtype: object

In [5]:
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    print(col,":",set(df[col].apply(type)))

company : {<class 'str'>}
location : {<class 'str'>}
industry : {<class 'str'>, <class 'float'>}
date : {<class 'str'>, <class 'float'>}
stage : {<class 'str'>, <class 'float'>}
country : {<class 'str'>}


Chúng ta nhận thấy rằng ba cột `industry`, `date` và `stage` có dạng dữ liệu không đồng nhất, tuy nhiên đây chỉ là do sự thiếu hụt dữ liệu trong các dòng, được biểu thị bằng giá trị `NaN` (dạng số thực).

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Các cột sau đang có kiểu dữ liệu là `object`, do đó cần được chuyển đổi sang kiểu chuỗi (`string`):

* `company`
* `location`
* `industry`
* `stage`
* `country`

Trong khi đó, cột `date` đang có kiểu dữ liệu là `object`, cần được chuyển đổi sang kiểu ngày tháng (`datetime`).

#### Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

In [6]:
# nếu cần tiền xử lí